<a href="https://colab.research.google.com/github/Alirs2002/fashion_GAN/blob/main/fashion_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [45]:
import tensorflow_datasets as tfds

In [46]:
data_set = tfds.load("fashion_mnist",split= "train")

In [47]:
def scale_images(data):
  img = data["image"]
  img = img/255
  return img

In [48]:
#data_set = data_set.as_numpy_iterator()

data_set= data_set.map(scale_images)
data_set = data_set.cache()
data_set = data_set.shuffle(60000)
data_set = data_set.batch(128)
data_set = data_set.prefetch(64)

In [49]:
data_set.as_numpy_iterator().next().shape

(128, 28, 28, 1)

now lets create our generator and the discriminator models

In [50]:
from tensorflow.keras import Sequential,Model

In [51]:
def create_generator():
  model = Sequential()

  model.add(tf.keras.layers.Dense(7*7*128,input_dim = 128))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Reshape((7,7,128)))

  model.add(tf.keras.layers.UpSampling2D())
  model.add(tf.keras.layers.Conv2D(128,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.UpSampling2D())
  model.add(tf.keras.layers.Conv2D(128,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2D(128,4,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2D(128,4,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2D(1,4,padding="same",activation="sigmoid"))

  return model



bellow is a function for creating discriminator

In [52]:
def create_discriminator():

  model = Sequential()

  model.add(tf.keras.layers.Input(shape=(28,28,1)))
  model.add(tf.keras.layers.Conv2D(32,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Conv2D(64,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Conv2D(128,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Conv2D(256,5,padding="same"))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Dropout(0.4))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dropout(0.4))
  model.add(tf.keras.layers.Dense(1,activation="sigmoid"))

  return model

In [53]:
generator = create_generator()
discriminator = create_discriminator()

now lets see a summary of the two parts of out GAN networks

In [54]:
generator.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 6272)              809088    
                                                                 
 leaky_re_lu_27 (LeakyReLU)  (None, 6272)              0         
                                                                 
 reshape_3 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d_6 (UpSamplin  (None, 14, 14, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 14, 14, 128)       409728    
                                                                 
 leaky_re_lu_28 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                      

In [55]:
discriminator.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 leaky_re_lu_32 (LeakyReLU)  (None, 28, 28, 32)        0         
                                                                 
 dropout_15 (Dropout)        (None, 28, 28, 32)        0         
                                                                 
 conv2d_33 (Conv2D)          (None, 28, 28, 64)        51264     
                                                                 
 leaky_re_lu_33 (LeakyReLU)  (None, 28, 28, 64)        0         
                                                                 
 dropout_16 (Dropout)        (None, 28, 28, 64)        0         
                                                                 
 conv2d_34 (Conv2D)          (None, 28, 28, 128)      

### now its time to create out training loop

In [56]:
g_opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
d_opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

g_loss = tf.keras.losses.BinaryCrossentropy()
d_loss = tf.keras.losses.BinaryCrossentropy()

In [57]:
class GAN_network(Model):
  def __init__(self,generator,discriminator,*args,**kwargs):
    super().__init__(*args,**kwargs)

    self.generator = generator
    self.discriminator = discriminator

  def compile(self,g_opt,d_opt,g_loss,d_loss):
    super.compile(*args,**kwargs)

    self.g_opt = g_opt
    self.d_opt = d_opt
    self.g_loss = g_loss
    self.d_loss = d_loss

  def train_step(self,batch):

    real_images = batch
    fake_images = self.generator(tf.random.normal((128,128,1)),training=False)

    with tf.GradientTape as tape:
      yhat_real = self.discriminator(real_images,training=True)
      yhat_fake = self.discriminator(fake_images,training=True)
      yhat_realfake = tf.concat([yhat_real,yhat_fake],axis = 0)

      y_real_fake = tf.concat([tf.zeros_like(yhat_real),tf.ones_like(yhat_fake)],axis = 0)

      noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
      noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
      y_realfake += tf.concat([noise_real,noise_fake],axis=0)

      total_d_loss = self.d_loss(y_realfake,yhat_realfake)

    d_gradient = tape.gradient(total_d_loss,self.discriminator.trainable_variables)
    self.d_opt.apply_gradients(zip(d_gradient,self.discriminator.trainable_variables))


    with tf.GradientTape as g_tape:

      generated_images = self.generator(tf.random.normal(128,128,1),training=True)
      predicted_labels = self.discriminator(generated_images,training=False)

      total_g_loss = self.g_loss(tf.zeros_like(predicted_labels),predicted_labels)

    g_gradient = g_tape(total_g_loss,self.generator.trainable_variables)
    self.g_opt.apply_gradients(zip(g_gradient,self.generator.trainable_variables))








